In [1]:
pip install pandas numpy sklearn nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libraries
import pandas as pd
import numpy as np
import re

In [3]:
BASE_PATH = "./dataset/"
# load dataset
df_metadata = pd.read_csv(BASE_PATH + "movies_metadata.csv")

/var/folders/69/qtpzcp6j1q9_ldjr9z9tpgkms6z_vx/T/ipykernel_20068/2382365070.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv(BASE_PATH + "movies_metadata.csv")


In [4]:
# filter columns
metadata_cols = ["id", "title", "overview", "tagline", "genres", "original_language", "poster_path"]
df_metadata   = df_metadata[metadata_cols]

# sort by id
df_metadata.sort_values(by=["id"], inplace=True)

In [5]:
# filter language
in_english     = df_metadata["original_language"] == "en"
df_metadata_en = df_metadata[in_english]

In [6]:
# parse names from genres column
genre_names                     = list(map(lambda g: re.findall("'name':\s*'(\w*)'", g), df_metadata_en["genres"]))
genre_names_joined              = list(map(lambda g: " ".join(g), genre_names))
df_metadata_en["genres_parsed"] = genre_names_joined

# combine features for tf-idf
df_metadata_en["document"] = (
    df_metadata_en["title"].astype(str) + " " + 
    df_metadata_en["overview"].astype(str) + " " + 
    df_metadata_en["tagline"].astype(str) + " " + 
    df_metadata_en["genres_parsed"].astype(str)
)

/var/folders/69/qtpzcp6j1q9_ldjr9z9tpgkms6z_vx/T/ipykernel_20068/1454195115.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata_en["genres_parsed"] = genre_names_joined
/var/folders/69/qtpzcp6j1q9_ldjr9z9tpgkms6z_vx/T/ipykernel_20068/1454195115.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata_en["document"] = (


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("omw-1.4")

lemmatizer = nltk.wordnet.WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# THIS CAN COME FROM TAGLINES
stop_words.add("nan")

[nltk_data] Downloading package punkt to /Users/jpturunen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jpturunen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jpturunen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jpturunen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
corpus = list(df_metadata_en["document"])
vocab  = {}

# clean corpus (remove stopwords, lemmatized etc.)
for i, doc in enumerate(corpus):
    as_lower     = doc.lower()
    tokens       = word_tokenize(as_lower)
    no_stopwords = [t for t in tokens if t not in stop_words]
    lemmatized   = [lemmatizer.lemmatize(t) for t in no_stopwords]
    corpus[i]    = " ".join(lemmatized)
    for t in lemmatized:
        if t not in vocab:
            vocab[t] = True
            
vocab = list(dict.keys(vocab))


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# calc tf-idf
tf_idf_vec = TfidfVectorizer(lowercase=False, stop_words=None, vocabulary=vocab, smooth_idf=True, use_idf=True)
tf_idf     = tf_idf_vec.fit_transform(corpus)

# print first doc
for i in range(5):    
    df = pd.DataFrame(tf_idf[i].T.todense(), index=tf_idf_vec.get_feature_names_out(), columns=["TF-IDF"])
    x = df.sort_values("TF-IDF", ascending=False).head(4)
    print(x)

## TODO: genres not appearing as often as I would like... maybe the tf-idf value should then always be 1 for those tokens?

               TF-IDF
sketchy      0.259745
unwillingly  0.247252
disgrace     0.238788
barrel       0.220365
             TF-IDF
einstein   0.370590
beer       0.304929
invention  0.298064
albert     0.296371
           TF-IDF
drake    0.360136
hellcat  0.283237
gray     0.219451
flight   0.191834
            TF-IDF
customer  0.603979
get       0.276022
simone    0.243164
mona      0.233355
                  TF-IDF
bloodwork       0.314088
allergy         0.277276
pharmaceutical  0.243626
testing         0.226059


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# similarity matrix
sim = cosine_similarity(tf_idf)

In [11]:
titles      = list(df_metadata_en["title"])
movie_title = "GoldenEye"
movie_idx   = titles.index(movie_title)
top_5_sim   = list(sim[movie_idx].argsort()[::-1][:5])

print(f"Top 5 movies most similar to {movie_title}:")
print("============================================")
for i in top_5_sim:
    print(f"- {titles[i]}")

Top 5 movies most similar to The Wolf of Wall Street:
- The Wolf of Wall Street
- The Toast of New York
- Wall Street: Money Never Sleeps
- Wall Street
- Wolves of Wall Street
